# Set Up

In [ ]:
%run ../setup.ipynb

In [ ]:
data_directory = '../DATA\\DSC2\\'
bucket_manifests_dir = 'S:\\Aymone\\serology\\bucket\\antibody_manifests\\'
bucket_returned_results_dir = 'S:\\Aymone\\serology\\bucket\\returned_results\\'

# Load data for the 19 seropositive and other pids

In [ ]:
the9 = pd.read_csv(data_directory+'the9_order.csv')#.iloc[:,1:]
the23 = pd.read_csv(data_directory+'the23_order.csv')#.iloc[:,1:]
the32 = pd.concat([the9, the23])[the9.columns].reset_index(drop = True)
the32['Biobank ID'] = ['A'+str(i) for i in the32.biobank_id]

the32

In [ ]:
the32.columns

## Load AB/EI, Orho/Roche Results of batch 1-5 and demographics

In [ ]:
master_list_original = pd.read_csv('../DATA\DSC2\Antobodies_tests_Master_List_2020-05-21.csv').drop_duplicates()[['participant_id', 'biobank_id', #'Age',
       'DateBloodSampleCollected', 'state', 'race', 'SexAtBirth',
       'DateOfBirth', 'Negative Control', 'Positive Control']]
master_list_original = master_list_original.merge(the32[['participant_id']])

master_list_original['Age'] = [math.floor(((pd.to_datetime(master_list_original['DateBloodSampleCollected'][i]) - pd.to_datetime(master_list_original['DateOfBirth'][i])).days)/365) for i in master_list_original.index]
master_list_original['Biobank ID'] = ['A'+str(i) for i in master_list_original.biobank_id]
master_list_original= master_list_original[['participant_id','biobank_id','DateOfBirth', 
                                           'race', 'participant_id', 'SexAtBirth', 'state', 'DateBloodSampleCollected']]

master_list_original


In [ ]:
# batch1to5_combined = pd.read_csv(data_directory+'batch1to5_combined.csv_2021-02-12.csv').merge(master_list_original, 'left').drop_duplicates()
# batch1to5_combined_demog = batch1to5_combined[['biobank_id', 'participant_id', 'Positive Control ID', 'Biobank ID',
#                    'batch', 'DateOfBirth', 
#                     'state', 'SexAtBirth', 'race', 'DateBloodSampleCollected',
#                     'Positive Control', 'Negative Control']]
# batch1to5_combined_demog.DateBloodSampleCollected = [datetime.date(pd.to_datetime(i)) for i in batch1to5_combined_demog.DateBloodSampleCollected]


In [ ]:
batch1to5_ab_ei_recalculated = pd.read_csv(data_directory+'batch1to5_ab_ei_recalculated_merged_1.12.2021.csv')#.iloc[:,1:]
batch1to5_ab_ei_recalculated = batch1to5_ab_ei_recalculated[['Biobank ID','biobank_id', "DateBloodSampleCollected",#'Positive Control ID', 
                               'Abbott;LIS Result_ab', 'EUROIMMUN;LIS Result_ei', 
                                'batch', 'od_ei','od_ab', 'outcome_ei', 'outcome_ab',
                                'Positive Control', 'Negative Control']]

In [ ]:
batch1to5_ab_ei_recalculated.columns

In [ ]:
selected_cols = ['Biobank ID', #'Patient Name', 
                 'Specimen ID', 
                 'COVTA COVTI',
       'COVTA COVTS', 'VCOV2 VSARS', 'VCOV2 SCO7', 
                # 'COVTI & VSARS match'
                ]

ortho_roche_results0 = pd.read_csv(bucket_returned_results_dir+'Roche and Ortho results_09212020_SNT_copy.csv')[selected_cols]
ortho_roche_results2 = pd.read_csv(bucket_returned_results_dir+'Roch_Ortho_Batch2_Results_copy_20201012.csv')[selected_cols]
ortho_roche_results2_0 = pd.read_csv(bucket_returned_results_dir+'antibody_manifests_Roche and Ortho results_09212020_SNT_copy.csv')[selected_cols]

#ortho_roche_results2['batch_test'] = 2
ortho_roche_results3 = pd.read_csv(bucket_returned_results_dir+'Roch_Ortho_Batch3_Results_copy_20201012.csv')[selected_cols]
ortho_roche_results4 = pd.read_csv(bucket_returned_results_dir+'antibody_manifests_Roch_Ortho_Batch4_Results - Copy.csv')[selected_cols]
#ortho_roche_results3['batch_test'] = 3
ortho_roche_results5 = pd.read_csv(bucket_returned_results_dir+'antibody_manifests_Roch_Ortho_Batch5_Results - Copy.csv')[selected_cols]


ortho_roche_results1 = pd.concat([ortho_roche_results2, ortho_roche_results3
                                  ,ortho_roche_results4, ortho_roche_results5
                                  #ortho_roche_results2_0, ortho_roche_results0
                                 ]).drop_duplicates()[selected_cols]


ortho_roche_results1 = ortho_roche_results1.drop('Specimen ID', 1).rename(columns = {'COVTA COVTI':'Roche Result',
                                                                         'COVTA COVTS':'Roche Score',
                                                                         'VCOV2 VSARS':'Ortho Result', 
                                                                          'VCOV2 SCO7':'Ortho Score',
                                                                         'COVTI & VSARS match':'Roche and Ortho match'})

ortho_roche_results1

In [ ]:
ortho_roche_results1[ortho_roche_results1['Biobank ID'] == "A136710369"]

In [ ]:
tuple(the32.participant_id.values)

In [ ]:
#batch1to5_elisas = ortho_roche_results1.merge(batch1to5_ab_ei_recalculated).drop_duplicates()
batch1to5_elisas = ortho_roche_results1.merge(batch1to5_ab_ei_recalculated)
batch1to5_elisas_the32 = batch1to5_elisas[batch1to5_elisas['Biobank ID'].isin(the32['Biobank ID'].values)]
#batch1to4_abei_ortho_roche.DateBloodSampleCollected = [datetime.date(pd.to_datetime(i)) for i in batch1to4_abei_ortho_roche.DateBloodSampleCollected]

In [ ]:
## a QNS in the 'Ortho Score' column is making drop_duplicates() not work
batch1to5_elisas_the32.loc[batch1to5_elisas_the32["Ortho Score"] == 'QNS', 'Ortho Result'] = 'QNS'
batch1to5_elisas_the32.loc[batch1to5_elisas_the32["Ortho Score"] == 'QNS', 'Ortho Score'] = None
batch1to5_elisas_the32["Ortho Score"] = batch1to5_elisas_the32["Ortho Score"].astype('float')
batch1to5_elisas_the32 = batch1to5_elisas_the32.reset_index().drop('index', 1).drop_duplicates()
batch1to5_elisas_the32

In [ ]:
batch1to5_elisas_the32.biobank_id.nunique()

In [ ]:
batch1to5_elisas_the32['One of the 32 Seropositive?']= 'Yes'

# Results in December and November 2020
and Check if all seropositives are in 

## Manifest

In [ ]:
ligia_manifest1 = pd.read_excel(bucket_manifests_dir+'antibody_manifests_PKG-2010-267457_AoU_NIH_10.28.2020.xlsx')
ligia_manifest1 = ligia_manifest1.rename(columns = {'Matrix (Tube) ID':'SID'})

df1 = ligia_manifest1[~ligia_manifest1['Biobank ID'].isnull()]
df1['biobank_id'] = [int(i.split('A')[1]) for i in df1['Biobank ID']]

# df = ligia_manifest1[ligia_manifest1['Biobank ID'].isnull()].reset_index().drop('index',axis = 1)
# for i in df.index:#['Biobank ID']:
#     df['Biobank ID'][i] = 'Positive Control '+str(i)

In [ ]:
ligia_manifest1 = df1# pd.concat([df1, df])
ligia_manifest1 = ligia_manifest1[['Biobank ID', 'biobank_id', 'Sample Id', 'SID','Study']]
ligia_manifest1

In [ ]:
ligia_manifest2 = pd.read_excel(bucket_manifests_dir+'NIH_Pinto_Serum_12.10.2020.xlsx')
ligia_manifest2 = ligia_manifest2.rename(columns = {'Sample ID':'Sample Id'})
ligia_manifest2 = ligia_manifest2.rename(columns = {'Matrix (Tube) ID':'SID'})

#ligia_manifest_all = pd.concat([ligia_manifest1,ligia_manifest2])
ligia_manifest2 = ligia_manifest2[~ligia_manifest2['Biobank ID'].isnull()]
ligia_manifest2['biobank_id'] = [int(i.split('A')[1]) for i in ligia_manifest2['Biobank ID']]
#ligia_manifest = ligia_manifest[['Biobank ID', 'biobank_id', 'SID','Collection Date','Study']]
ligia_manifest2

In [ ]:
ligia_manifest3 = pd.read_excel(bucket_manifests_dir+'Titers_LigiaLab_2.12.2021.xlsx')#[['Biobank ID','Sample Id', 'SID']]
ligia_manifest3['biobank_id'] = [int(i.split('A')[1]) for i in ligia_manifest3['Biobank ID']]
#ligia_manifest = ligia_manifest[['Biobank ID', 'biobank_id', 'SID','Collection Date','Study']]
ligia_manifest3

## Merge Manifest with Results

In [ ]:
ligia_file1 = bucket_returned_results_dir+'Mayo Clinic _results_sent06NOV2020_v2_ligia.xlsx'
ligia_file2 = bucket_returned_results_dir+'Mayo Clinic _results_23DEC20_V2.xlsx'
ligia_file3 = bucket_returned_results_dir+'Mayo Clinic _results_12MAR21_vFINAL.xlsx'

In [ ]:
pd.read_excel(ligia_file2, sheet_name = 4).head(1)

In [ ]:
ligia_manifest_all.head(1)

In [ ]:
pd.read_excel(ligia_file1, sheet_name = 1).head(1)

In [ ]:
def get_ligia_results(which_file, manifest_df):
    
    ligia_results_spikeIgM = manifest_df.merge(pd.read_excel(which_file, sheet_name = 1))
    display(ligia_results_spikeIgM)
    ligia_results_spikeIgM = ligia_results_spikeIgM[['biobank_id','Spike IgM (AU/mL)']]
    print('Spike IgM Results for:'+str(ligia_results_spikeIgM.biobank_id.nunique())+' people.')
    
    ligia_results_spikeIgG = manifest_df.merge(pd.read_excel(which_file, sheet_name = 2))
    ligia_results_spikeIgG = ligia_results_spikeIgG[['biobank_id','Spike IgG (AU/mL)']]
    print('Spike IgG Results for:'+str(ligia_results_spikeIgG.biobank_id.nunique())+' people.')
    
    ligia_results_nuclIgG = manifest_df.merge(pd.read_excel(which_file, sheet_name = 3))
    ligia_results_nuclIgG = ligia_results_nuclIgG[['biobank_id','Nucleocapsid IgG (AU/mL)']]
    print('nuclIgG Results for:'+str(ligia_results_nuclIgG.biobank_id.nunique())+' people.')
    
    ligia_results_nuclIgM = manifest_df.merge(pd.read_excel(which_file, sheet_name = 4))
    ligia_results_nuclIgM = ligia_results_nuclIgM[['biobank_id','Nucleocapsid IgM (AU/mL)']]
    print('nuclIgM Results for:'+str(ligia_results_nuclIgM.biobank_id.nunique())+' people.')
    
    ligia_results_all = ligia_results_spikeIgM.merge(ligia_results_spikeIgG).merge(ligia_results_nuclIgG).merge(ligia_results_nuclIgM)
    print('\nTotal Results for:'+str(ligia_results_all.biobank_id.nunique())+' people.')
    
    return ligia_results_all

In [ ]:
get_ligia_results(ligia_file2, ligia_manifest2)

In [ ]:
ligia_results1 = get_ligia_results(ligia_file1, ligia_manifest1)
ligia_results2 = get_ligia_results(ligia_file2, ligia_manifest2)
ligia_results3 = get_ligia_results(ligia_file3, ligia_manifest3)
ligia_results1to3 = pd.concat([ligia_results1, ligia_results2, ligia_results3])#.merge(batch1to5_elisas_the32[['biobank_id','One of the 32 Seropositive?']], 'left')
ligia_results1to3.biobank_id.nunique()

In [ ]:
#batch1to5_elisas['Abbott;LIS Result_ab'] = batch1to5_elisas['Abbott;LIS Result_ab'].astype('string')

In [ ]:
batch1to5_elisas[~batch1to5_elisas['biobank_id'].isin(ligia_results1to3['biobank_id'].values)].drop_duplicates()


In [ ]:
batch1to5_elisas_the76 = batch1to5_elisas[batch1to5_elisas['biobank_id'].isin(ligia_results1to3['biobank_id'].values)].drop_duplicates()
batch1to5_elisas_the76.biobank_id = batch1to5_elisas_the76.biobank_id.astype('int64') #[['Roche Result','','','']]
batch1to5_elisas_the76 = batch1to5_elisas_the76[(batch1to5_elisas_the76['Negative Control'] == 'No') & 
                                                (batch1to5_elisas_the76['Positive Control'] == 'No')].drop(['outcome_ab'
                                                                                                            ,'outcome_ei'
                                                                                                            ,'Biobank ID',
                                                                                                            'Positive Control','Negative Control'
                                                                                                           ],1)

In [ ]:
batch1to5_elisas_the76.loc[batch1to5_elisas_the76['Ortho Score'] == 'QNS', 'Ortho Score'] = None
#theQNS = batch1to5_elisas_the77[batch1to5_elisas_the77['Ortho Score'] == 'QNS']
batch1to5_elisas_the76[['Roche Result','Ortho Result']] = batch1to5_elisas_the76[['Roche Result','Ortho Result']].astype('string')
batch1to5_elisas_the76['Ortho Score'] = batch1to5_elisas_the76['Ortho Score'].astype('float64')

In [ ]:
batch1to5_elisas_the76.drop_duplicates()

In [ ]:
batch1to5_elisas_the76[batch1to5_elisas_the76.biobank_id == 293960432]

In [ ]:
mysterious76_perShawn = pd.read_csv(data_directory+'Mysterious-76.csv')
mysterious76_perShawn_pids = mysterious76_perShawn[['Biobank.ID']].drop_duplicates()
mysterious76_perShawn_pids.columns = ['biobank_id']
mysterious76_perShawn_pids

In [ ]:
all_76_elisas = ligia_results1to3.merge(batch1to5_elisas_the76, 'right')
all_76_elisas.drop_duplicates()#[all_77_elisas['Ortho Result'] == 'QNS']

In [ ]:
all_76_elisas.to_csv(data_directory+'the76_with_titers_and_elisas.csv')

In [ ]:
all_76_elisas[['biobank_id','Abbott;LIS Result_ab','Ortho Result','Roche Result', 'EUROIMMUN;LIS Result_ei']].groupby(['Abbott;LIS Result_ab','Ortho Result','Roche Result', 'EUROIMMUN;LIS Result_ei']).nunique()

In [ ]:
len(set(all_76_elisas.biobank_id) - set(the32.biobank_id))

## QC

### Specimens sent to Ligia's Lab not of the 32

In [ ]:
in_ligia_lab = pd.DataFrame(set(the32.biobank_id).intersection(set(ligia_results1to3.biobank_id)),
                                columns = ['biobank_id']).merge(batch1to5_elisas_the32) #ligia_manifest2
in_ligia_lab

In [ ]:
set(the32.biobank_id) in set(ligia_results1to3.biobank_id)

In [ ]:
the32_ligia_lab = the32.merge(ligia_results1to3)
the32_ligia_lab

In [ ]:
the32_ligia_lab.biobank_id.nunique()

In [ ]:
the32_ligia_lab.to_csv(data_directory+'the32_ligia_lab.csv', index = False)

In [ ]:
in_ligia_lab[in_ligia_lab.biobank_id == 977982923]

In [ ]:
not_in_ligia_lab = pd.DataFrame(set(the32.biobank_id) - set(ligia_results1_and_2.biobank_id),
                                columns = ['biobank_id']).merge(batch1to5_elisas_the32) #ligia_manifest2
not_in_ligia_lab

In [ ]:
27+5

`325704569` is a negative control that was sent to Ligia's lab

### Those of the 19 that Mayo did not send

In [ ]:
of_the19_notsent = the19_order[['Order','biobank_id']].merge(pd.DataFrame(set(the19.biobank_id) - set(ligia_results1_allpeople.biobank_id), columns = ['biobank_id']).dropna().merge(the19_tests))
of_the19_notsent.columns

In [ ]:
of_the19_notsent[['Order','batch','biobank_id', 'DateOfBirth',
       'state', 'SexAtBirth', 'race', 'DateBloodSampleCollected',
        'Roche Result', 'Ortho Result', 'Abbott Result',
       'EUROIMMUN Result', 'outcome_ei', 'outcome_ab', 'batch', 
       'Positive Control', 'Negative Control', 
       'One of the 19 Seropositive?']]

# Results on January 2021

## Manifest

In [ ]:
ligia_manifest2 = pd.read_excel(bucket_manifests_dir+'NIH_Pinto_Serum_12.10.2020.xlsx')
ligia_manifest2 = ligia_manifest2.rename(columns = {'Sample ID':'Sample Id'})
ligia_manifest2 = ligia_manifest2.rename(columns = {'Matrix (Tube) ID':'SID'})
ligia_manifest2['biobank_id'] = [int(i.split('A')[1]) for i in ligia_manifest2['Biobank ID']]
ligia_manifest2 = ligia_manifest2[['Biobank ID', 'biobank_id', 'Sample Id', 'SID']]

# ligia_manifest_all = pd.concat([ligia_manifest1,ligia_manifest2])
# ligia_manifest = ligia_manifest_all[~ligia_manifest_all['Biobank ID'].isnull()]
ligia_manifest2

## Merge Manifest with Results

In [ ]:
ligia_file2 = bucket_returned_results_dir+'Mayo Clinic _results_23DEC20_V2.xlsx'

In [ ]:
ligia_results2_merged = get_ligia_results(which_file = ligia_file2, manifest_df = ligia_manifest2)

# all people in the results
ligia_results2_allpeople = ligia_results2_merged.merge(batch1to4_abei_ortho_roche_all)
ligia_results2_allpeople.DateBloodSampleCollected = [datetime.date(pd.to_datetime(i)) for i in ligia_results2_allpeople.DateBloodSampleCollected]
print('All people results: '+str(ligia_results2_allpeople.biobank_id.nunique()))

# from the 19 
ligia_results2_the19 = ligia_results2_allpeople[ligia_results2_allpeople['One of the 19 Seropositive?'] == 'Yes']
ligia_results2_the19 = the19_order[['Order','biobank_id']].merge(ligia_results2_the19)
print('From the 19 seropositives: '+str(ligia_results2_the19.biobank_id.nunique()))

# Other specimens
ligia_results2_theothers = ligia_results2_allpeople[ligia_results2_allpeople['One of the 19 Seropositive?'] == 'No']
print('All other results: '+str(ligia_results2_theothers.biobank_id.nunique()))

## QC

### Specimens sent to Ligia's Lab not of the 19

In [ ]:
ls = tuple(set(ligia_results2_allpeople.biobank_id) - set(the19.biobank_id))
ls

In [ ]:
non_in_the19 = ligia_results2_allpeople[ligia_results2_allpeople.biobank_id.isin(ls)]
non_in_the19.columns

In [ ]:
non_in_the19 = non_in_the19.replace({'POSITIVE':1, 'Positive':1, 'NEGATIVE':0,"Negative":0})

In [ ]:
non_in_the19[['biobank_id', #'participant_id',
              'Spike IgM (AU/mL)', 'Spike IgG (AU/mL)',
       'Nucleocapsid IgG (AU/mL)', 'Nucleocapsid IgM (AU/mL)', 
       'Roche Result', 'Roche Score', 'Ortho Result','Ortho Score', 
        'outcome_ab','od_ab', 'outcome_ei', 'od_ei',
       #'Positive Control ID', 'batch', #'DateOfBirth', 
        'state', 'SexAtBirth','race', 'DateBloodSampleCollected', 
              #'Positive Control',
       #'Negative Control', 'One of the 19 Seropositive?'
             ]]

In [ ]:
#non_in_the19[non_in_the19.participant_id == 893884192]


In [ ]:
non_in_the19[['Negative Control','Positive Control']].drop_duplicates()

In [ ]:
non_in_the19['Age'] = [math.floor(((pd.to_datetime(non_in_the19['DateBloodSampleCollected'][i]) - pd.to_datetime(non_in_the19['DateOfBirth'][i])).days)/365) for i in non_in_the19.index]

In [ ]:
non_in_the19[['biobank_id', 'Spike IgM (AU/mL)', 'Spike IgG (AU/mL)',
       'Nucleocapsid IgG (AU/mL)', 'Nucleocapsid IgM (AU/mL)', 'Roche Result', 'Roche Score', 'Ortho Result',
       'Ortho Score', 'DateBloodSampleCollected', 'Age', 'state', 'SexAtBirth',
       'race',  'outcome_ab','od_ab', 'outcome_ei', 'od_ei']].to_csv(data_directory+'Others_Ligia_lab.csv', index = False)

### Those of the 19 that Mayo did not send

In [ ]:
of_the19_notsent = the19_order[['Order','biobank_id']].merge(pd.DataFrame(set(the19.biobank_id) - set(ligia_results2_allpeople.biobank_id) - set(ligia_results1_the19.biobank_id), columns = ['biobank_id']).dropna().merge(the19_tests))
of_the19_notsent#.columns

In [ ]:
ligia_results_the19 = pd.concat([ligia_results1_the19, ligia_results2_the19])
ligia_results_the19

In [ ]:
ligia_results_the19.to_csv(data_directory+'the19_Ligia_lab.csv', index = False)

# QC on 10.25.21 using file Issac/Sheri sent

## Al manifest combined

In [ ]:
all_manisfests = pd.concat([ligia_manifest1, ligia_manifest2, ligia_manifest3])[['Biobank ID','biobank_id','Sample Id', 'SID']]
all_manisfests

## Merge Manifest with File Isaac Sent

In [ ]:
all_samples_sent_to_lab = pd.read_excel(data_directory+'serology to Ligia.xlsx', sheet_name='Sheet2')
all_samples_sent_to_lab = all_samples_sent_to_lab[['Sample Id','Origin','Collection Date']].drop_duplicates()

noncontrols_sent_to_lab = all_samples_sent_to_lab[all_samples_sent_to_lab.Origin == 'PMI Participant']
noncontrols_sent_to_lab

In [ ]:
all_samples_sent_to_lab[all_samples_sent_to_lab.Origin != 'PMI Participant'].Origin.unique()

In [ ]:
all_samples_sent_to_lab['Sample Id'].nunique()

In [ ]:
ligia_manifest_all = noncontrols_sent_to_lab[['Sample Id']].merge(all_manisfests)
ligia_manifest_all

## Merge with Results

In [ ]:
ligia_file2

In [ ]:
ligia_results1_merged = get_ligia_results(which_file = ligia_file1, manifest_df = ligia_manifest_all)
ligia_results2_merged = get_ligia_results(which_file = ligia_file2, manifest_df = ligia_manifest_all)
ligia_results3_merged = get_ligia_results(which_file = ligia_file3, manifest_df = ligia_manifest_all)

In [ ]:
ligia_results_merged = pd.concat([ligia_results1_merged,ligia_results2_merged,ligia_results3_merged])
ligia_results_merged

## mETGING WITHTHE OTHER RESULTS

In [ ]:
ligia_results_merged_results = ligia_results_merged.merge(batch1to5_elisas_the76).drop_duplicates()
ligia_results_merged_results

In [ ]:
set(ligia_results_merged.biobank_id) - set(ligia_results_merged_results.biobank_id)

In [ ]:
ligia_results_merged_results = ligia_manifest_all[['Sample Id','biobank_id']]\
.merge(ligia_results_merged_results[['biobank_id', 'DateBloodSampleCollected','Spike IgM (AU/mL)', 'Spike IgG (AU/mL)',
       'Nucleocapsid IgG (AU/mL)', 'Nucleocapsid IgM (AU/mL)', 'Roche Result',
       'Roche Score', 'Ortho Result', 'Ortho Score', 'Abbott;LIS Result_ab','od_ab',
       'EUROIMMUN;LIS Result_ei', 'od_ei', 'batch']]).drop('biobank_id',1)

ligia_results_merged_results['control_status'] = 'Non Control'

In [ ]:
ligia_results_merged_results

In [ ]:
the1_missing = ligia_manifest_all[['biobank_id','Sample Id']].merge(
    ligia_results_merged.merge(batch1to5_elisas[batch1to5_elisas.biobank_id.isin({325704569})]))#.drop_duplicates()).drop('biobank_id',1)
the1_missing = the1_missing[['Sample Id', 'DateBloodSampleCollected','Spike IgM (AU/mL)', 'Spike IgG (AU/mL)',
       'Nucleocapsid IgG (AU/mL)', 'Nucleocapsid IgM (AU/mL)', 'Roche Result',
       'Roche Score', 'Ortho Result', 'Ortho Score', 'Abbott;LIS Result_ab','od_ab',
       'EUROIMMUN;LIS Result_ei', 'od_ei', 'batch']]

the1_missing

In [ ]:
the1_missing = the1_missing[['Sample Id', 'DateBloodSampleCollected','Spike IgM (AU/mL)', 'Spike IgG (AU/mL)',
       'Nucleocapsid IgG (AU/mL)', 'Nucleocapsid IgM (AU/mL)', 'Roche Result',
       'Roche Score', 'Ortho Result', 'Ortho Score', 'Abbott;LIS Result_ab','od_ab',
       'EUROIMMUN;LIS Result_ei', 'od_ei', 'batch']].drop_duplicates().iloc[:2]

the1_missing['control_status'] = 'Negative Control'
the1_missing

In [ ]:
all_ligia_results_merged_results = pd.merge(ligia_results_merged_results, the1_missing, 'outer').drop_duplicates().reset_index()
all_ligia_results_merged_results

In [ ]:
data_directory

In [ ]:
all_ligia_results_merged_results.to_csv(data_directory+'all_ligia_results_merged_results.csv', index  = False)